In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [18]:
# Konstante
SENSEBOX_ID="6793f4e4c326f20007c34dd2"
API_URL_FORMAT_BOX="https://api.opensensemap.org/boxes/{sensebox_id}?format={response_format}"
API_URL_FORMAT_SENSOR="https://api.opensensemap.org/boxes/{sensebox_id}/data/{sensor_id}?from-date={fromDate}&to-date={toDate}&download=true&format=json"

# Ein dict erzeugen mit {sensor name : sensor id} und ausgeben lassen
def get_sensor_names_ids():
    url = API_URL_FORMAT_BOX.format(sensebox_id=SENSEBOX_ID,response_format="json")
    status_code = requests.get(url).status_code
    assert status_code , f"Failed fetching data from api {status_code}"
    
    sensors = requests.get(url).json().get("sensors")
    sensor_name_id = {}
    for sensor in sensors:
        name = sensor.get('title').replace(" ","").replace(".","_").replace("-","_")
        _id = sensor.get('_id')
        sensor_name_id.update({name : _id})
    return sensor_name_id

### So kann man die namen und ids als eine Liste haben

In [17]:
get_sensor_names_ids().keys(), get_sensor_names_ids().values()

(dict_keys(['Temperatur', 'rel_Luftfeuchte', 'Beleuchtungsstärke', 'UV_Intensität', 'PM1', 'PM2_5', 'PM4', 'PM10']),
 dict_values(['6793f4e4c326f20007c34dd3', '6793f4e4c326f20007c34dd4', '6793f4e4c326f20007c34dd5', '6793f4e4c326f20007c34dd6', '6793f4e4c326f20007c34dd7', '6793f4e4c326f20007c34dd8', '6793f4e4c326f20007c34dd9', '6793f4e4c326f20007c34dda']))

### Dict Ausgabe von der Funktion

In [10]:
get_sensor_names_ids()

{'Temperatur': '6793f4e4c326f20007c34dd3',
 'rel_Luftfeuchte': '6793f4e4c326f20007c34dd4',
 'Beleuchtungsstärke': '6793f4e4c326f20007c34dd5',
 'UV_Intensität': '6793f4e4c326f20007c34dd6',
 'PM1': '6793f4e4c326f20007c34dd7',
 'PM2_5': '6793f4e4c326f20007c34dd8',
 'PM4': '6793f4e4c326f20007c34dd9',
 'PM10': '6793f4e4c326f20007c34dda'}

### So kann man auf die einzelne Namen zugreifen

In [4]:
sensor_id = get_sensor_names_ids()["Temperatur"]
sensor_id

'6793f4e4c326f20007c34dd3'

### create_dataframe function mit inputs sensor_id und from- und toDate die einen default Wert haben

In [14]:
def create_dataframe(sensor_id, fromDate="2025-04-10T00:00:00Z", toDate="2025-04-24T00:00:00Z"):
    url = API_URL_FORMAT_SENSOR.format(sensebox_id=SENSEBOX_ID, sensor_id=sensor_id, fromDate=fromDate, toDate=toDate)
    
    status_code = requests.get(url).status_code
    assert status_code == 200, f"Failed fetching data from api {status_code}"
    
    data = requests.get(url).json()
    return pd.DataFrame(data)

In [15]:
create_dataframe(sensor_id=get_sensor_names_ids()["rel_Luftfeuchte"])

,location,createdAt,value
0,"[-48.480915, -1.299151]",2025-04-23T23:59:26.693Z,96.27
1,"[-48.480915, -1.299151]",2025-04-23T23:58:26.555Z,96.96
2,"[-48.480915, -1.299151]",2025-04-23T23:57:26.346Z,96.56
3,"[-48.480915, -1.299151]",2025-04-23T23:56:25.902Z,96.56
4,"[-48.480915, -1.299151]",2025-04-23T23:55:25.714Z,96.47
...,...,...,...
9995,"[-48.480915, -1.299151]",2025-04-16T22:18:29.056Z,97.71
9996,"[-48.480915, -1.299151]",2025-04-16T22:17:29.008Z,97.75
9997,"[-48.480915, -1.299151]",2025-04-16T22:16:35.293Z,97.41
9998,"[-48.480915, -1.299151]",2025-04-16T22:15:28.203Z,97.55
